# By-Band _g_-Point Reduction

# Dependencies

`numpy` is installed in the Python environment at NERSC (`module load python`), but `xarray` is not, so the user must install the package on their own. `PIPPATH` is the assumed location. This notebook depends heavily on `xarray`.

In [1]:
import os, sys, shutil, glob

# "standard" install
import numpy as np

# directory in which libraries installed with conda are saved
PIPPATH = '{}/.local/'.format(os.path.expanduser('~')) + \
    'cori/3.7-anaconda-2019.10/lib/python3.7/site-packages'
PATHS = ['common', PIPPATH]
for path in PATHS: sys.path.append(path)

# user must do `pip install xarray` on cori (or other NERSC machines)
import xarray as xa

# local module
import by_band_lib as BYBAND

# Static Inputs

In [2]:
# only do one domain or the other
DOLW = True
DOSW = not DOLW
DOMAIN = 'lw' if DOLW else 'sw'
NBANDS = 16 if DOLW else 14

# forcing scenario (0 is no forcing...need a more comprehensive list)
IFORCING = 0

# does band-splitting need to be done, or are there existing files 
# that have divided up the full k-distribution?
BANDSPLIT = False

# remove the netCDFs that are generated for all of the combinations 
# and iterations of combinations in bandOptimize()
CLEANUP = False

# Paths

In [3]:
PROJECT = '/global/project/projectdirs/e3sm/pernak18/'
KFULLNC = '{}/reference_netCDF/g-point-reduce/'.format(PROJECT) + \
  'rrtmgp-data-lw-g256-2018-12-04.nc'
EXE = '{}/g-point-reduction/garand_atmos/rrtmgp_garand_atmos'.format(
    PROJECT)
GARAND = '{}/reference_netCDF/g-point-reduce/'.format(PROJECT) + \
  'multi_garand_template.nc'
PATHS = [KFULLNC, EXE]

BANDSPLITDIR = 'band_k_dist'
FULLBANDFLUXDIR = 'full_band_flux'
if not BANDSPLIT: PATHS += [BANDSPLITDIR, FULLBANDFLUXDIR]

for PATH in PATHS: BYBAND.pathCheck(PATH)

CWD = os.getcwd()

# Band Splitting

Break up full _k_-distribution file into separate distributions for each band, then calculate the corresponding fluxes. This should only need to be run once.

After some clarifications from Robert (30-Nov-2020), I believe the plan of action is:

1. Perform _g_-point combination for 1 band at a time
2. Combine condensed _k_-distribution for a given band with the full distributions from the rest of the bands
3. Run RRTMGP on output from item 2
4. Compute cost function for broadband fluxes, not by-band

Original Plan of Action:

- [] We would create Nbands k-distribution files
- [] We’d use Python to drive the Fortran executable Nbands times to produce Nbands flux results
- [] The trial g-point combinations then loop over bands and the possible g-point combinations within each band, creating k-distribution and band-wise flux files for each possible combination
- [] The Python code assembles broadband fluxes from the band-wise flux files in order to compute the cost functions

In [4]:
if BANDSPLIT:
    # divide full k-distribution into subsets for each band
    print('Band splitting commenced')
    kFiles = BYBAND.kDistBandSplit(KFULLNC, 
        outDir=BANDSPLITDIR, domain=DOMAIN)
    print('Band splitting completed')

    # quick, non-parallelized flux calculations (because the 
    # executable is run in one directory)
    print('Full band flux calculations commenced')
    BYBAND.pathCheck(FULLBANDFLUXDIR, mkdir=True)
    fullBandFluxes = []
    for kFile in kFiles:
        BYBAND.fluxCompute(kFile, 
            fluxDir=FULLBANDFLUXDIR, atmSpecFile=GARAND)
        fullBandFluxes.append(outFile)
    # end kFile loop
    print('Full band flux calculations completed')
else:
    kFiles = sorted(glob.glob('{}/coefficients_{}_band??.nc'.format(
        BANDSPLITDIR, DOMAIN)))
    fullBandFluxes = sorted(glob.glob('{}/flux_{}_band??.nc'.format(
        FULLBANDFLUXDIR, DOMAIN)))
# endif BANDSPLIT


# _g_-Point Combining

Loop over bands and combine their respective _g_-points. For each combination in a band, compute fluxes, then concatenate these fluxes with the full-band fluxes from other bands, then compute broadband fluxes and corresponding cost function.

In [5]:
# loop over bands and instantiate a band optimization object

# leave `bands` empty if all bands should be processed
bands = [1]
if not bands: bands = range(1, NBANDS+1)

# loop over bands and instantiate a band optimization object
# optimizing each band
for iBand, kFile in enumerate(kFiles):
    band = iBand + 1
    if band not in bands: continue

    BYBAND.bandOptimize(kFile, band, DOLW, IFORCING, fullBandFluxes, 
                        cleanup=CLEANUP)
    print('Band {} complete'.format(band))
# end kFile loop


Starting Band 1, iteration 1
Starting Band 1, iteration 2
Starting Band 1, iteration 3
Starting Band 1, iteration 4
Starting Band 1, iteration 5
Starting Band 1, iteration 6
Starting Band 1, iteration 7
Starting Band 1, iteration 8
Starting Band 1, iteration 9
Starting Band 1, iteration 10
Starting Band 1, iteration 11
Starting Band 1, iteration 12
Starting Band 1, iteration 13
Starting Band 1, iteration 14
Starting Band 1, iteration 15
Starting Band 1, iteration 16
Band 1 complete
